<a href="https://colab.research.google.com/github/kasparvonbeelen/lancaster-newspaper-workshop/blob/wc/LLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U "transformers==4.40.0" datasets --upgrade

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import transformers
from datasets import Dataset
import torch
from tqdm import tqdm
import pandas as pd
import torch

device = 'cuda'

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/kasparvonbeelen/lancaster-newspaper-workshop/wc/data/subsample500mixedocr-selected_mitch.csv')
df.head(3)

,publication_code,issue_id,item_id,newspaper_title,data_provider,date,year,month,day,location,word_count,ocrquality,political_leaning_label,price_label,text
0,2249,624,art0017,The Bee-Hive.,British Library Heritage Made Digital Newspapers,1871-06-24,1871,6,24,"London, England",271,0.9098,liberal,1d,"THE TICHBORNE CASE. On Tuesday, before the So..."
1,2250,908,art0002,"The Industrial Review, Social and Political.",British Library Heritage Made Digital Newspapers,1877-09-08,1877,9,8,"London, England",2791,0.9841,liberal,2d,THE CLERGY AND TRADE UNIONS. LETTER FROM REV....
2,2250,406,art0024,"The Industrial Review, Social and Political.",British Library Heritage Made Digital Newspapers,1878-04-06,1878,4,6,"London, England",304,0.9870,liberal,2d,INDUSTRIAL REVIEW OUR LEGISLATORS. THE unrul...


In [ ]:
df = df[df.word_count.between(10,250)].reset_index()
df.shape

(212, 16)

In [ ]:
# make an account on hugging face https://huggingface.co/
# go to https://huggingface.co/meta-llama/Meta-Llama-3-8B
# and accept the licence. it may take a few minutes to get a reply!

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
checkpoint = "meta-llama/Meta-Llama-3-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=checkpoint,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device="cuda",
)

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
messages = [
    {"role": "system", "content": """
    You are an helpful AI that will assist me with analysing and reading newspaper articles.
    Read the newspaper articles attentively and extract the required information.
    Each newspaper article will be enclosed with triple hash tags (i.e. ###).
    Don't make thigs up! If the information is not in the article then just say 'Dunno'"""},
    {"role": "user", "content": f"""Provide a short description of principal characters portrayed newspaper article?
    ###{df.iloc[0].text}###"""}
]


In [ ]:
def get_completion(messages, temperature=.1, top_p=.1):

  prompt = pipeline.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
      )

  outputs = pipeline(
    prompt,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=temperature,
    top_p=top_p,
      )
  return outputs[0]["generated_text"][len(prompt):]

In [ ]:
print(get_completion(messages))

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Here is a short description of the principal characters portrayed in the newspaper article:

1. The mate of the steam tug "Earl of Glamorgan" - The unfortunate man who drowned in the Severn a few days ago and is the subject of the article.
2. The parish undertaker - The person responsible for arranging the funeral and burial of the deceased.
3. The brother of the deceased - The person who is disputing the expense of the funeral and burial and is refusing to release the body until his claims are met.


## Exercise

- Change the system message and ask the model to reply in Swiss-German
- Change the user message and ask the model to summarize the article and condense it to one sentence

In [ ]:
messages = [
    {"role": "system", "content": """
    You are an helpful AI that will assist me with analysing and reading newspaper articles.
    Read the newspaper articles attentively and extract the required information.
    Each newspaper article will be enclosed with triple hash tags (i.e. ###).
    Don't make thigs up! Answer in Swiss-German!"""},
    {"role": "user", "content": f"""Provide a short description of principal characters portrayed newspaper article?
    ###{df.iloc[0].text}###"""}
]

print(get_completion(messages))


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Ja, wat a seltsame Geschichte! (What a strange story!) Let me tell you about de principal characters:

* De unfortunate mate of de steam tug Earl of Glamorgan, who got drowned in de Severn a few days ago. He's de one who's gettin' buried, but his identity wasn't known at first.
* De parish undertaker, who got de order to make a parish coffin for de mate. He's de one who's responsible for de funeral arrangements.
* De brother of de deceased, who's gettin' all upset about de expensive coffin and wants to know who's gonna pay for it.
* De coroner, who's tryin' to mediate de situation but seems to be powerless.

So, it's a story about a funeral gone wrong, with some drama and conflict between de undertaker, de brother, and de coroner.


In [ ]:
messages = [
    {"role": "system", "content": """
    You are an helpful AI that will assist me with analysing and reading newspaper articles.
    Read the newspaper articles attentively and extract the required information.
    Each newspaper article will be enclosed with triple hash tags (i.e. ###).
    Don't make thigs up! If the information is not in the article then just say 'Dunno'"""},
    {"role": "user", "content": f"""Summarize the article content in one sentence.
    ###{df.iloc[0].text}###"""}
]

print(get_completion(messages))

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


A body believed to be the mate of the steam tug Earl of Glamorgan, who drowned in the Severn, was initially intended for a parish coffin but was instead given a more expensive one, leading to a dispute over who should pay for the funeral.


## Example: Summarization

In [ ]:
df_small = df.sample(5, random_state=1984).reset_index(drop=True)
df_small

,index,publication_code,issue_id,item_id,newspaper_title,data_provider,date,year,month,day,location,word_count,ocrquality,political_leaning_label,price_label,text
0,209,3077,512,art0058,"Nelson Chronicle, Colne Observer, and Clithero...",British Library Living with Machines Project,1899-05-12,1899,5,12,"Nelson, Lancashire, England",160,0.9748,liberal,1d,"NEW FASHIONS. BELL (V, SON, 28 & 30, MANCHES..."
1,352,3040,1210,art0150,The Birkenhead News and Wirral General Adverti...,British Library Living with Machines Project,1910-12-10,1910,12,10,"Birkenhead, Merseyside, England",105,0.8768,conservative,½ d<SEP>1d,LOANS To THE COUNCIL. At a meeting of the Liv...
2,227,3089,426,art0036,Glasgow Courier.,British Library Living with Machines Project,1855-04-26,1855,4,26,"Glasgow, Strathclyde, Scotland",97,0.9647,conservative,3 ½ d<SEP>4 ½ d,J. K. DONALD & W. NEVILLE B- • - • ---EG to i...
3,293,2974,312,art0014,"The Stourbridge Observer, Cradley Heath, Hales...",British Library Living with Machines Project,1887-03-12,1887,3,12,"Stourbridge, West Midlands, England",13,0.7369,NaN,NaN,"TOBACCOS, AT TEX OLE ESTABLISH EI) S ITO FIE..."
4,449,3074,818,art0103,The North-Eastern Weekly Gazette.,British Library Living with Machines Project,1894-08-18,1894,8,18,"Stockton-on-Tees, Cleveland, England",139,0.8834,liberal,½ d,IT OF CHILDREN YOLIFFE. ice Court on Tuesday...


In [ ]:

def apply_completions(item, system_message, user_message):
  messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_message}
      ]
  messages[1]['content'] += f"\n\n###{item['text']}###"
  return  get_completion(messages)

In [ ]:
tqdm.pandas()

system_message = """You are an helpful AI that will assist me with analysing and reading newspaper articles.
    Read the newspaper articles attentively and extract the required information.
    Each newspaper article will be enclosed with triple hash tags (i.e. ###).
    Don't make thigs up! If the information is not in the article then just say 'Dunno'"""
user_message = "Summarize the article content in one sentence."

df_small['completion'] =  df_small.progress_apply(apply_completions,system_message=system_message, user_message=user_message, axis=1)


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [ ]:
df_small['completion']

0    Here is a summary of the article content in on...
1    The Livetpoul City Council considered a recomm...
2    Here is a summary of the article content in on...
3    The article announces the establishment of a n...
4    Here is a summary of the article in one senten...
Name: completion, dtype: object

## Example: Biography as microgenre

In [ ]:
df_small = df.sample(10, random_state=1984).reset_index(drop=True)

In [ ]:
system_message = """You are an helpful AI that will assist me with analysing and reading newspaper articles.
    Read the newspaper articles attentively and extract structured information.
    Each newspaper article will be enclosed with triple hash tags (i.e. ###).
    Don't make thigs up!"""


user_message = """Who are the characters portrayed in the article?
    Extract biographical from a newspaper article.
    For each identified person return a nested Python dictionary with the key equal to the name of the individual.
    The values conist of dictionaries that record specific attributes such as age, gender, nationality, profession ,place of birth etc.
    The format has to be a Python dictionary, do not add extra text!"""

In [ ]:
df_small['completion'] =  df_small.progress_apply(apply_completions,system_message=system_message, user_message=user_message, axis=1)


100%|██████████| 10/10 [01:06<00:00,  6.61s/it]


In [ ]:
df_small['completion'][5]

"Here is the extracted information in a Python dictionary format:\n\n{\n    'Rev. Mr. Rhodes': {\n        'profession': 'clergy',\n        'nationality': 'British'\n    },\n    'Mr. John Haling': {\n        'profession': 'Offlay Arnie',\n        'nationality': 'British'\n    },\n    'Mr. William Steele': {\n        'profession': 'Flitch of Bacon',\n        'nationality': 'British'\n    },\n    'Mr. Reeves': {\n        'profession': 'host',\n        'nationality': 'British'\n    },\n    'Mrs. Reeves': {\n        'profession': 'hostess',\n        'nationality': 'British'\n    },\n    'Warburton': {\n        'profession':'surgeon',\n        'nationality': 'British'\n    },\n    'Palmer': {\n        'profession':'surgeon',\n        'nationality': 'British'\n    }\n}"

In [ ]:
eval(df_small['completion'][5].split('format:\n\n')[-1].strip())

{'Rev. Mr. Rhodes': {'profession': 'clergy', 'nationality': 'British'},
 'Mr. John Haling': {'profession': 'Offlay Arnie', 'nationality': 'British'},
 'Mr. William Steele': {'profession': 'Flitch of Bacon',
  'nationality': 'British'},
 'Mr. Reeves': {'profession': 'host', 'nationality': 'British'},
 'Mrs. Reeves': {'profession': 'hostess', 'nationality': 'British'},
 'Warburton': {'profession': 'surgeon', 'nationality': 'British'},
 'Palmer': {'profession': 'surgeon', 'nationality': 'British'}}

In [ ]:
eval(df_small['completion'][4].split('format:\n\n')[-1].strip())

{'Robert Thompson': {'age': None,
  'gender': None,
  'nationality': None,
  'profession': 'S.P.C.C. Inspector',
  'place_of_birth': 'Aycliffe'},
 'Mr J. T. Proud': {'age': None,
  'gender': None,
  'nationality': None,
  'profession': 'S.P.C.C. Inspector',
  'place_of_birth': None}}

In [ ]:
eval(df_small['completion'][2].split('format:\n\n')[-1].strip())

{'J. K. Donald': {'name': 'J. K. Donald',
  'profession': 'Watchmaker and Jeweller'},
 'W. Neville': {'name': 'W. Neville', 'profession': 'Watchmaker and Jeweller'}}

## Example: OCR correction

In [ ]:
df_small_bad_ocr = df.sort_values('ocrquality')[:5]

In [ ]:
user_message = """Transcribe the text and correct typos and errors in the text caused by bad optical character recognition (OCR).
Do not add any information that is not in the original text!"""

df_small_bad_ocr['completion'] = df_small_bad_ocr.progress_apply(apply_completions,system_message=system_message, user_message=user_message, axis=1)


100%|██████████| 5/5 [00:30<00:00,  6.08s/it]


In [ ]:
print(df_small_bad_ocr.iloc[0]['text'])

4.9,oUMPLlAZill—dorr Walnut. annininiiire ‘.7 in nacallent condition. a Ibn. 1.41:.ND; Rigiten porkatzili tu!l drawn . 


In [ ]:
print(df_small_bad_ocr.iloc[0]['completion'])

I'd be happy to help!

Here is the transcribed text with corrections:

###4.9, Multiplex Mall - Door Walnut, anniversary.7 in excellent condition. A Ibn. 1.41: AND; Righten porkatzili till drawn. ###


In [ ]:
print(df_small_bad_ocr.iloc[4]['text'])

MaEWAN & WALLACH.  itulilliSUA KILL). EMITS AGICtiTa LAN' VALUERS - v 34, H AMILTUN squARE, 1.7b-lopti 1:4 ul6  AkA;ll  BAND AND SMITH ESTATE AlitE,NTd, SUBNEWILS V AlAi MS, 71. LORD-STILLET LIVFX.POOI4 Div= th4truusa Qrle LibrelWaoots LIMP POOL sod DlValot. Wine, r Idiom i4colli*Mor4di. Tolpphoos WIT Biak. 391 NEVI OliarrialieliOAD, Rock Parry.— &ask etabia or akau Yard 8.401 3s. 6d  WEls. KIRBY A/CD HOYLAXE 7ar - OE LET OR SOLD AMY TO W. F. B"v-"• ESTArE AGENT AND VALUER 3, GRANGE ROAD, wEer ILEXBY Telopkrme Hoyaak• 89. gry1684.1.7  Itia•bllabed 1/0. QU FAN AND FOSTER  INSTATE AGENTS & SAMBAS. 2 8013TH STICSAT,LIVSEPOOL Warsaw " ti4c.o4 TellebOute. Zink 4ii6 1177:1  J0H.30  


In [ ]:
print(df_small_bad_ocr.iloc[4]['completion'])

Here is the transcribed text with corrections for typos and OCR errors:

###MaEWAN & WALLACH. It is said that Maewan & Wallach will kill). EMITS AGENT LAN' VALUERS - 34, H AMILTON SQUARE, 1.7b-loft 1:4 ul6  AkA;ll  BAND AND SMITH ESTATE ALIENATED, SUBNEWILS VALUATION, 71. LORD-STILET LIVELY. POOL AND DVALOT. Wine, red or white, is included. Top floor WIT Bank. 391 NEW Oriel Road, Rock Ferry.— Ask estate agent or auction yard 8.401 3s. 6d. WEls. KIRBY AND HOYLAXE 7ar - OE LET OR SOLD AMY TO W. F. Bov-"• ESTATE AGENT AND VALUER 3, GRANGE ROAD, WEER ILEXBY Telephone Hoylake 89. Gry1684.1.7. It is believed 1/0. QU FAN AND FOSTER INSTATE AGENTS & SAMBAS. 2 8013TH STICSAT, LIVSE


In [ ]:
df_small_bad_ocr.to_csv('newspaper_ocr_corrected.csv')

## Combing filtering and targeted prompting

In [ ]:
import re
pattern = re.compile(r'\baccident[s]{0,1}\b',re.I)
df_kw_sample = df[df.apply(lambda x: bool(pattern.findall(x.text)), axis=1)]

user_message = """Does the newspaper describe a historical accident? If not return an empty Python list'.
If it does describe an accident extract, information on the people involved in the accident.
Return a list of Python dictionaries. For each dictionary the key is equal to the name of the person.
The values list charactertistics of this person such a gender, age and occupation.
Only return the Python list and no additional text!
"""
df_kw_sample['completion'] = df_kw_sample.progress_apply(apply_completions, user_message=user_message, system_message=system_message, axis=1)
df_kw_sample.to_csv('accidents.csv')

100%|██████████| 3/3 [00:05<00:00,  1.91s/it]


In [ ]:
df_kw_sample['completion']

51    [\n    {"Chadder": {"gender": "male", "age": "...
79                                                   []
80    [\n    {'Postman': {'gender':'male', 'age': 'u...
Name: completion, dtype: object

In [ ]:
eval(df_kw_sample.iloc[0]['completion'])

[{'Chadder': {'gender': 'male',
   'age': 'unknown',
   'occupation': 'naval reserves'}},
 {'James Edmund Flood': {'gender': 'male',
   'age': '18',
   'occupation': 'unknown'}}]

## Exercise

Experiment with your own system and user message.